In [1]:
import os, sys
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')



In [2]:
read_data = pd.read_csv('2019_4_2_02379003-6369-43bf-b444-842420685d06.csv')


In [3]:
read_data.columns

Index(['clientId', 'clientIncome', 'incomeVerified', 'clientAge',
       'clientGender', 'clientMaritalStatus', 'clientLoanPurpose',
       'clientResidentialStauts', 'clientState', 'clientTimeAtEmployer',
       'clientNumberPhoneContacts', 'clientAvgCallsPerDay', 'loanType',
       'loanNumber', 'applicationDate', 'approvalDate', 'declinedDate',
       'disbursementDate', 'payout_status', 'dueDate', 'paidAt', 'loanAmount',
       'interestRate', 'loanTerm', 'max_amount_taken', 'max_tenor_taken',
       'paymentRatio', 'FirstPaymentDefault', 'loanDefault'],
      dtype='object')

In [4]:
read_data['incomeVerified'][read_data['incomeVerified'] == 'true'] = 1
read_data['incomeVerified'][read_data['incomeVerified'] == 'false'] = 0

In [5]:
read_data = read_data.drop('loanType', axis = 1)

In [6]:
read_data['clientResidentialStauts'][read_data['clientResidentialStauts']== 'Null'] = 'Rented'

In [7]:
read_data['clientState'][read_data['clientState'] == 'Null'] = 'LAGOS'
read_data['clientState'].value_counts()

LAGOS          60667
OGUN           14798
ABUJA          13999
OYO            12731
RIVERS          7829
DELTA           5081
KWARA           3969
OSUN            3816
ONDO            3571
KADUNA          3496
EDO             2663
NIGER           2462
BENUE           2043
EKITI           2023
AKWA IBOM       1960
KOGI            1896
PLATEAU         1715
CROSS RIVER     1608
NASARAWA        1589
ENUGU           1566
ANAMBRA         1496
ABIA            1426
IMO             1287
KANO            1233
BAYELSA         1179
ADAMAWA          569
BAUCHI           418
SOKOTO           398
EBONYI           392
TARABA           336
KEBBI            279
GOMBE            265
KATSINA          248
ZAMFARA          221
BORNO            210
YOBE              83
JIGAWA            72
OJO                1
LAGOS              1
Name: clientState, dtype: int64

In [8]:
read_data['clientMaritalStatus'].value_counts()

Married      85492
Single       71361
Separated     1795
Widowed        939
Null             6
Divorced         3
Name: clientMaritalStatus, dtype: int64

In [9]:
read_data['clientMaritalStatus'][read_data['clientMaritalStatus']== 'Null'] = 'Married'

In [10]:
read_data['clientLoanPurpose'].value_counts()

business     78976
other        31554
house        22240
education    15737
medical      11089
Name: clientLoanPurpose, dtype: int64

In [11]:
read_data['clientResidentialStauts'].value_counts()

Rented               101670
Own Residence         26406
Family Owned          25668
Employer Provided      5589
Temp. Residence         263
Name: clientResidentialStauts, dtype: int64

In [12]:
data = pd.get_dummies(read_data,prefix_sep = '_', columns = ['clientGender', 'clientState', 'clientMaritalStatus',
                                         'clientLoanPurpose', 'clientResidentialStauts'])

In [13]:
data.head(3)

,clientId,clientIncome,incomeVerified,clientAge,clientTimeAtEmployer,clientNumberPhoneContacts,clientAvgCallsPerDay,loanNumber,applicationDate,approvalDate,...,clientLoanPurpose_business,clientLoanPurpose_education,clientLoanPurpose_house,clientLoanPurpose_medical,clientLoanPurpose_other,clientResidentialStauts_Employer Provided,clientResidentialStauts_Family Owned,clientResidentialStauts_Own Residence,clientResidentialStauts_Rented,clientResidentialStauts_Temp. Residence
0,719046128,25000.00,1,38,36+,2976,51.40909090909091,4,2018-03-18,2018-03-18,...,1,0,0,0,0,0,0,0,1,0
1,821565030,105000.00,0,31,36+,1159,121.03645833333333,4,2018-01-06,2018-01-06,...,0,0,1,0,0,0,0,0,1,0
2,703822576,78029.19,1,38,36+,1375,13.402912621359222,4,2018-04-27,2018-04-27,...,0,1,0,0,0,0,1,0,0,0


In [14]:
data = data.drop(['applicationDate', 'approvalDate', 'declinedDate',
       'disbursementDate', 'payout_status', 'dueDate', 'paidAt'], axis = 1)

In [15]:
x = data.drop(['clientId', 'clientNumberPhoneContacts'], axis = 1)

In [16]:
for k in x.columns:
    value = sum([isinstance(i, str) for i in x[k]])
    if value > 1:
        print(k, value)
    else:
        pass

incomeVerified 103
clientTimeAtEmployer 159596
clientAvgCallsPerDay 159596


In [17]:
x['incomeVerified'].value_counts()

0       133286
1        26207
Null       103
Name: incomeVerified, dtype: int64

In [18]:
x['incomeVerified'][x['incomeVerified']== 'Null'] = 0

In [19]:
x['clientTimeAtEmployer'][x['clientTimeAtEmployer']== 'Null'] = 36

In [20]:
x = x.drop(['clientTimeAtEmployer', 'clientAvgCallsPerDay'], axis = 1)

In [21]:
'loanDefault' in x.columns

True

In [24]:
target = x['loanDefault']
x = x.drop('loanDefault', axis = 1)
target.shape

(159596,)

In [32]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

min_max = MinMaxScaler()
std = StandardScaler()
norm = Normalizer()

x_new = min_max.fit_transform(x)
x_std = std.fit_transform(x_new)
x_norm = std.fit_transform(x_std)

In [33]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_norm, target, test_size = 0.4, random_state = 42)

x_train_dataframe = pd.DataFrame(x_train)
x_test_dataframe = pd.DataFrame(x_test)
y_train_dataframe = pd.DataFrame(y_train)
y_test_dataframe = pd.DataFrame(y_test)


x_train_dataframe.to_csv('x_train_nn.csv')
x_test_dataframe.to_csv('x_test_nn.csv')
y_train_dataframe.to_csv('y_train_nn.csv')
y_test_dataframe.to_csv('y_test_nn.csv')

# feature selection

In [25]:
from sklearn.feature_selection import RFE

from sklearn.linear_model import LogisticRegression
# feature extraction
model = LogisticRegression()
rfe = RFE(model, 3)
fit = rfe.fit(x, target)
print('Num Features: %d' % fit.n_features_)
print('Selected Features: %s' % fit.support_)
print('Feature Ranking: %s' % fit.ranking_)

Num Features: 3
Selected Features: [False False False False False False False False False  True  True  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False]
Feature Ranking: [51  4 50 39 53 45 48 38  3  1  1  1  2 19 28 29 24 59 16 18 57 20 23 31 60
 26 43 37 36 54 56 30 22 62  5 41 49 27 64 34 17 33 63 40 46 42 25 32 58 44
 61 55 65 47 35 21 52 14 13 12 15 11  6 10  9  8  7]


In [26]:
for i, j in zip(fit.support_, x.columns):
    if i == True:
        print(j)
    else:
        pass

paymentRatio
FirstPaymentDefault
clientGender_FEMALE


In [27]:
new_feature = x[['paymentRatio', 'FirstPaymentDefault','clientGender_FEMALE']]

In [28]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(new_feature, target, test_size = 0.4, random_state = 42)


In [29]:
x_train_dataframe = pd.DataFrame(x_train)
x_test_dataframe = pd.DataFrame(x_test)
y_train_dataframe = pd.DataFrame(y_train)
y_test_dataframe = pd.DataFrame(y_test)


x_train_dataframe.to_csv('x_train.csv')
x_test_dataframe.to_csv('x_test.csv')
y_train_dataframe.to_csv('y_train.csv')
y_test_dataframe.to_csv('y_test.csv')